Model with transfer learning using the VGG-16 architecture, data preprocessing with data augmentation and classifier with 2 Dense hidden layers with Dropout.
Fit of the model implemented with early stopping.


In [ ]:
#Imports
import json 
import tensorflow as tf
import numpy as np
import os
import pandas as pd
from datetime import datetime
from os import listdir
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#Set seed to ensure experiment replication
SEED = 1234
tf.random.set_seed(SEED)

# Get current working directory
cwd = os.getcwd()


In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
#Connect to Drive to fetch data
from google.colab import drive
drive.mount('/content/drive') 


In [ ]:
#unzip the data
!unzip '/content/drive/My Drive/artificial-neural-networks-and-deep-learning-2020.zip'

In [ ]:
#Create Data Generators
#Training data augmentation to training data Toggle

apply_data_augmentation = True

# Create training ImageDataGenerator object
if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(shear_range=0.2,
                                        width_shift_range=0.15,
                                        height_shift_range=0.15,
                                        zoom_range=0.3,
                                        horizontal_flip=True,
                                        validation_split=0.2,
                                        fill_mode='constant',
                                        cval=0,
                                        rescale=1./255)
else:
    train_data_gen = ImageDataGenerator(rescale=1./255,
                                        validation_split=0.2)

# Create validation and test ImageDataGenerator objects
valid_data_gen = ImageDataGenerator(rescale=1./255)
test_data_gen = ImageDataGenerator(rescale=1./255)




dataset_dir = os.path.join(cwd, "MaskDataset")
training_dir= os.path.join(dataset_dir,"training")

#Batch size
BS = 8

#Image shape
IMG_H = 407 
IMG_W = 407

num_classes=3


with open(os.path.join(dataset_dir,"train_gt.json")) as f:
  dic=json.load(f)
  dataframe= pd.DataFrame(dic.items())
  dataframe.rename(columns={0:'filename', 1:'class'}, inplace=True )
  dataframe=dataframe.sample(frac=1, random_state=SEED)
  dataframe["class"]=dataframe["class"].astype('string')
  train_gen = train_data_gen.flow_from_dataframe(dataframe,
                                                 training_dir,
                                                 batch_size=BS,
                                                 target_size=(IMG_H, IMG_W),
                                                 class_mode='categorical',
                                                 shuffle= True,
                                                 seed=SEED,
                                                subset='training',
                                                image_size=(407,407))
  
  validation_gen = train_data_gen.flow_from_dataframe(dataframe,
                                                 training_dir,
                                                 batch_size=BS,
                                                 target_size=(IMG_H, IMG_W),
                                                 class_mode='categorical',
                                                 shuffle= True,
                                                 seed=SEED,
                                                 subset='validation',
                                                image_size=(407,407))
  




Found 4492 validated image filenames belonging to 3 classes.
Found 1122 validated image filenames belonging to 3 classes.


In [ ]:
#create Dataset Object

#Training
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, IMG_H, IMG_W, 3], [None, num_classes]))

train_dataset = train_dataset.repeat()

#Validation
valid_dataset = tf.data.Dataset.from_generator(lambda: validation_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, IMG_H, IMG_W, 3], [None, num_classes]))

valid_dataset = valid_dataset.repeat()


In [ ]:
#VGG model
vgg = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(IMG_H, IMG_W, 3))
vgg.summary()

58892288/58889256 [==============================] - 0s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 407, 407, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 407, 407, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 407, 407, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 203, 203, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 203, 203, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 203, 203, 128)     147584    
______________________________________________________________

In [ ]:
# Architecture: VGG -> Classifier

#Finetuning model
finetuning = True

if finetuning:
    freeze_until = 15 # layer from which we want to fine-tune
    
    for layer in vgg.layers[:freeze_until]:
        layer.trainable = False
else:
    vgg.trainable = False

#Create model
model = tf.keras.Sequential() #we implement this in a sequential way 
model.add(vgg)

# Classifier
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=256, activation='relu'))
model.add(tf.keras.layers.Dropout(0.1,seed=SEED))
model.add(tf.keras.layers.Dense(units=128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2,seed=SEED))
model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

In [ ]:
# Visualize created model as a table
model.summary()

# Visualize initialized weights
#model.weights

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 12, 12, 512)       14714688  
_________________________________________________________________
flatten (Flatten)            (None, 73728)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               18874624  
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 3

In [ ]:
#Optimization parameters:

# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 1e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

# Validation metrics
metrics = ['accuracy']
# ------------------

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
#create Callbacks

cwd = os.getcwd()

exps_dir = os.path.join('/content/drive/My Drive/Keras/', 'classification_experiments')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

model_name = 'First'

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks = []

# Model checkpoint
ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'), 
                                                   save_weights_only=True)  # False to save the model directly
callbacks.append(ckpt_callback)


#Implementation of Early Stopping
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10,restore_best_weights = True )
    callbacks.append(es_callback)

In [ ]:
model.fit(x=train_dataset,
          epochs=40,  #### set repeat in training dataset
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(validation_gen),
          callbacks=callbacks)


Epoch 1/40
562/562 [==============================] - 297s 529ms/step - loss: 0.8436 - accuracy: 0.5717 - val_loss: 0.6597 - val_accuracy: 0.6488
Epoch 2/40
562/562 [==============================] - 296s 526ms/step - loss: 0.5012 - accuracy: 0.7745 - val_loss: 0.4354 - val_accuracy: 0.8102
Epoch 3/40
562/562 [==============================] - 299s 533ms/step - loss: 0.4355 - accuracy: 0.8139 - val_loss: 0.3672 - val_accuracy: 0.8449
Epoch 4/40
562/562 [==============================] - 311s 554ms/step - loss: 0.3771 - accuracy: 0.8459 - val_loss: 0.4658 - val_accuracy: 0.8048
Epoch 5/40
562/562 [==============================] - 313s 558ms/step - loss: 0.3361 - accuracy: 0.8629 - val_loss: 0.3141 - val_accuracy: 0.8743
Epoch 6/40
562/562 [==============================] - 313s 558ms/step - loss: 0.3110 - accuracy: 0.8753 - val_loss: 0.3520 - val_accuracy: 0.8494
Epoch 7/40
562/562 [==============================] - 313s 557ms/step - loss: 0.2630 - accuracy: 0.9036 - val_loss: 0.3172 -

In [ ]:
#Make prediction on test data and export it as csv file

#Create_csv function definition
def create_csv(results, results_dir='./'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')

#import test data
test_dir = os.path.join(dataset_dir, 'test')

images = [f for f in listdir(test_dir)]
images = pd.DataFrame(images)
images.rename(columns = {0:'filename'}, inplace = True)
images["class"] = 'test'

test_gen = train_data_gen.flow_from_dataframe(images,
                                               test_dir,
                                               batch_size=BS,
                                               target_size=(IMG_H, IMG_W),
                                               class_mode='categorical',
                                               shuffle=False,
                                               seed=SEED)


test_gen.reset()

#create predictions
predictions = model.predict_generator(test_gen, len(test_gen), verbose=1)

results = {}
images = test_gen.filenames
i = 0

for p in predictions:
  prediction = np.argmax(p)
  import ntpath
  image_name = ntpath.basename(images[i])
  results[image_name] = str(prediction)
  i = i + 1





In [ ]:
create_csv(results, '/content/drive/My Drive')